In [1]:
import numpy as np
import metpy

from metpy.calc import dewpoint_from_relative_humidity, mixed_layer_cape_cin
from metpy.units import units
import scipy.optimize as so
import scipy.integrate as si
np.set_printoptions(suppress=True)

# Constants:

In [2]:
gamma = 6.5 # K/km
p0 = 1000 # hPa
p0_stp = 1013.25 # hPa
t0 = 288. # K
Rd = 287.05 # J/kg/K
depth = 100 # hPa
epsilon = 0.62196
sat_press_0c = 6.112 # hPa
kappa = 0.285714
g = 9.81 # m/s^2
Lv = 2500840 # J/kg
Cp_d = 1004.6662184201462 # J/kgK

# Function Definitions:

In [3]:
def mixing_ratio(partial_press, total_press):
    return epsilon * partial_press / (total_press - partial_press)

def vapor_pressure(pressure, mixing_ratio):
    # pressure in hPa, mixing_ratio in kg/kg
    return pressure * mixing_ratio / (epsilon + mixing_ratio)

def saturation_vapor_pressure(temperature):
    # temperature in celsius 
    return sat_press_0c * np.exp(17.67 * temperature / (temperature + 243.5))

def exner_function(pressure):
    # pressure in hPa
    return (pressure / p0)**kappa

def get_pressure_height(pressure):
    pressure = np.atleast_1d(pressure)
    height =  (t0 / gamma) * (1 - (pressure / p0)**(Rd * gamma / g))
    return pressure, height

def potential_temperature(temperature, pressure):
    # assume incoming temp in celsius, output in kelvin
    theta = (temperature+273.15)/ exner_function(pressure)
    return theta

def virtual_temperature(temperature, mixing_ratio):
    # temperature in kelvin, mixing_ratio in kg/kg
    return temperature * (1 + 0.61 * mixing_ratio) / (1 + mixing_ratio)

def dewpoint(vapor_pressure):
    val = np.log(vapor_pressure / sat_press_0c)
    return 243.5 * val / (17.67 - val)

def dry_lapse(pressure, temperature):
    return temperature * (pressure / pressure[0])**kappa

def saturation_mixing_ratio(pressure, temperature):
    return mixing_ratio(saturation_vapor_pressure(temperature), pressure)

cape_cin steps:
1. calculate lcl using lowest level (p, t, td)
2. get the mixing ratio of the parcel to get the parcel profile
3. calculate the virtual temperature from (p, t, td)
4. get the parcel profile through using the virtual temperature function (?) using parcel_profile and parcel_mixing_ratio
5. calculate lfc pressure using (p, t_v, td, new_parcel_profile, bottom "which_lfc")
6. calculate el pressure using (p, t_v, td, new_parcel_profile, top "which_el"), if nan, use top of pressure
7. find y = (parcel_profile - temperature) in kelvin
8. Use pressure and y to find zero crossings, returning x and y
9. Using data between LFC and EL, use Rd * trapezoid integration method of y and log(x) to get CAPE
10. Can do CIN in the future with this method by using between surface and LFC instead (negative value or 0 expected)

### Initial datasets:

In [4]:
p = [1008., 1000., 950., 900., 850., 800., 750., 700., 650., 600.,
550., 500., 450., 400., 350., 300., 250., 200.,
175., 150., 125., 100., 80., 70., 60., 50.,
40., 30., 25., 20.] * units.hPa
# temperature
T = [29.3, 28.1, 25.5, 20.9, 18.4, 15.9, 13.1, 10.1, 6.7, 3.1,
-0.5, -4.5, -9.0, -14.8, -21.5, -29.7, -40.0, -52.4,
-59.2, -66.5, -74.1, -78.5, -76.0, -71.6, -66.7, -61.3,
-56.3, -51.7, -50.7, -47.5] * units.degC
# relative humidity
rh = [.85, .75, .56, .39, .82, .72, .75, .86, .65, .22, .52,
 .66, .64, .20, .05, .75, .76, .45, .25, .48, .76, .88,
 .56, .88, .39, .67, .15, .04, .94, .35] * units.dimensionless
# calculate dewpoint
Td = dewpoint_from_relative_humidity(T, rh)


In [5]:
# Convert p and T to numpy arrays for easier manipulation
p_array = p.magnitude
T_array = T.magnitude
Td_array = Td.magnitude

# Initialize the 5D array with zeros
data_5d = np.zeros((2, 180, 360, len(p), 3))

# Fill the array with the same values for all grid points
# This is just sample data - in a real application, values would vary by location
for i in range(2):  # Loop through first dimension
    for j in range(180):  # Loop through latitude dimension
        for k in range(360):  # Loop through longitude dimension
            # Assign pressure values to the first channel of the last dimension
            data_5d[i, j, k, :, 0] = p_array
            # Assign temperature values to the second channel of the last dimension
            data_5d[i, j, k, :, 1] = T_array
            # Assign dewpoint values to the third channel of the last dimension
            data_5d[i, j, k, :, 2] = Td_array

# Print the shape to confirm
print(f"Shape of the 5D array: {data_5d.shape}")

Shape of the 5D array: (2, 180, 360, 30, 3)


## Mixed parcel reverse engineer

### Metpy:

In [6]:
expected_parcel_pressure, expected_parcel_temp, expected_parcel_dewpoint = metpy.calc.mixed_parcel(p, T, Td)


### Rewritten test:

In [7]:
# Calculate theta, assume hPa:
theta = potential_temperature(T_array, p_array)
theta_mp = metpy.calc.potential_temperature(p, T)
print(theta)
# Calculate saturation mixing ratio:
es = saturation_vapor_pressure(Td_array)
mixing_ratio_g_g = mixing_ratio(es, p_array)

[301.7622202  301.25       303.05900772 303.03635908 305.40704209
 308.07859228 310.77225675 313.63720657 316.5035492  319.65947389
 323.43537383 327.48795335 331.84299536 335.66456569 339.6746038
 343.40258237 346.45943129 349.62801786 352.03587038 355.33474197
 360.56837652 375.81006569 405.6946951  430.87813976 461.2264245
 498.59848266 543.96437507 603.09185398 638.20966737 690.00915612]


#### Mixed Layer:

##### Metpy:

In [8]:
output= metpy.calc.mixed_layer(p, theta_mp, mixing_ratio_g_g*metpy.units.units.dimensionless, depth=100*metpy.units.units.hPa)
depth_mp = 100*metpy.units.units.hPa

##### Rewritten Test:

In [9]:
# get_layer: pressure, theta, mixing_ratio_g_kg, p_array[0], no height, no depth (ml is 100 hpa), no interpolate

bottom_pressure, bottom_height = get_pressure_height(np.atleast_1d(p_array[0]))
top = bottom_pressure - depth # hPa
top_pressure, top_height = get_pressure_height(top)

# Subset pressure levels between bottom and top with some margin
# Add a small margin to ensure we capture the full layer
pressure_mask = (p_array >= (top_pressure[0])) & (p_array <= (bottom_pressure[0]))

p_interp = p_array[pressure_mask]
if not np.any(np.isclose(top_pressure, p_interp)):
        p_interp = np.sort(np.append(p_interp, top_pressure))
if not np.any(np.isclose(bottom_pressure, p_interp)):
        p_interp = np.sort(np.append(p_interp, bottom_pressure))


p_interp = p_interp[::-1]
layer_depth = abs(p_interp[0] - p_interp[-1])
interps = []
ret = []
for vars in [theta, mixing_ratio_g_g]:
    vars_interp = metpy.calc.tools.log_interpolate_1d(p_interp, p_array, vars) # need to get out of metpy
    interps.append(vars_interp)
    integration = np.trapezoid(vars_interp, p_interp)/-layer_depth
    ret.append(integration)

interp_theta = interps[0]
interp_mixing_ratio = interps[1]
mean_theta = ret[0]
mean_mixing_ratio = ret[1]

print(f'expected: {output}')
print(f'actual: {ret}' )

expected: [<Quantity(302.47855, 'kelvin')>, <Quantity(0.0133943232, 'dimensionless')>]
actual: [np.float64(302.47854625268184), np.float64(0.013394323196384561)]


#### Rest of mixed_parcel:

In [10]:
theta_back_to_temp = (mean_theta) * exner_function(p_array[0])

vapor_pres = vapor_pressure(p_array[0], mean_mixing_ratio)
vapor_pres_back_to_dewpoint = dewpoint(vapor_pres)

print(p_array[0])
print(theta_back_to_temp - 273.15)
print(vapor_pres_back_to_dewpoint)


1008.0
30.017958708208937
18.474902116257926


In [11]:
print(expected_parcel_pressure)
print(expected_parcel_temp)
print(expected_parcel_dewpoint)

1008.0 hectopascal
30.017963177354034 degree_Celsius
18.47490211625791 degree_Celsius


## Append outputs from mixed_parcel to truncated arrays

In [12]:
start_p = p[0]
# Remove values below top of mixed layer and add in the mixed layer values
pressure_prof = p[p < (start_p - depth_mp)]
temp_prof = T[p < (start_p - depth_mp)]
dew_prof = Td[p < (start_p - depth_mp)]
pressure_prof_mp = metpy.units.concatenate([expected_parcel_pressure, pressure_prof])
temp_prof_mp = metpy.units.concatenate([expected_parcel_temp, temp_prof])
dew_prof_mp = metpy.units.concatenate([expected_parcel_dewpoint, dew_prof])


In [13]:
# Remove values below top of mixed layer and add in the mixed layer values
pressure_prof = p_array[p_array < (p_array[0] - depth)]
temp_prof = T_array[p_array < (p_array[0] - depth)]
dew_prof = Td_array[p_array < (p_array[0] - depth)]
pressure_prof = np.concatenate([np.atleast_1d(p_array[0]), pressure_prof])
temp_prof = np.concatenate([np.atleast_1d(theta_back_to_temp - 273.15), temp_prof])
dew_prof = np.concatenate([np.atleast_1d(vapor_pres_back_to_dewpoint), dew_prof])


## parcel_profile_with_lcl

### Metpy:

In [14]:
temp_prof_mp

<Quantity([ 30.01796318  20.9         18.4         15.9         13.1
  10.1          6.7          3.1         -0.5         -4.5
  -9.         -14.8        -21.5        -29.7        -40.
 -52.4        -59.2        -66.5        -74.1        -78.5
 -76.         -71.6        -66.7        -61.3        -56.3
 -51.7        -50.7        -47.5       ], 'degree_Celsius')>

In [15]:
p_wLCL, T_wLCL, Td_wLCL, prof_wLCL = metpy.calc.parcel_profile_with_lcl(pressure_prof_mp, temp_prof_mp, dew_prof_mp)
print("p_wLCL: ", p_wLCL)
print("T_wLCL: ", T_wLCL)
print("Td_wLCL: ", Td_wLCL)
print("prof_wLCL: ", prof_wLCL)

[852.2779038467842 850.0 800.0 750.0 700.0 650.0 600.0 550.0 500.0 450.0 400.0 350.0 300.0 250.0 200.0 175.0 150.0 125.0 100.0 80.0 70.0 60.0 50.0 40.0 30.0 25.0 20.0] hectopascal 288.975249885237 kelvin
temp_upper [288.975249885237 288.8771556652491 286.63455860650197 284.2002456235547 281.5375457747217 278.59863205450785 275.31986372934097 271.6147430262509 267.3633103953191 262.3965121588944 256.4753716364084 249.27086166014854 240.3665451455124 229.3133143497968 215.67473493183098 207.7092353100362 198.80834257354783 188.734751014561 177.08204579851522 166.14516860910948 159.92588765957808 153.0351360864167 145.26732675049576 136.29482269942747 125.54012629833461 119.16791491037053 111.80745110731084] kelvin
p_wLCL:  [1008.0 900.0 852.2779038467842 850.0 800.0 750.0 700.0 650.0 600.0 550.0 500.0 450.0 400.0 350.0 300.0 250.0 200.0 175.0 150.0 125.0 100.0 80.0 70.0 60.0 50.0 40.0 30.0 25.0 20.0] hectopascal
T_wLCL:  [30.017963177354034 20.9 18.513895192339206 18.4 15.9 13.1 10.1 6.7

### Rewritten test:

#### lcl:

In [16]:
metpy.calc.lcl(pressure_prof_mp[0], temp_prof_mp[0], dew_prof_mp[0])


(<Quantity(852.277904, 'hectopascal')>,
 <Quantity(15.8252499, 'degree_Celsius')>)

In [17]:
print(pressure_prof_mp[0])
print(temp_prof_mp[0])
print(dew_prof_mp[0])
print(pressure_prof[0])
print(temp_prof[0])
print(dew_prof[0])

1008.0 hectopascal
30.017963177354034 degree_Celsius
18.47490211625791 degree_Celsius
1008.0
30.017958708208937
18.474902116257926


In [18]:
#pressure needs to be in Pa, temperature and dewpoint needs to be in K

pressure_prof_pa = pressure_prof*100
temp_prof_k = temp_prof+273.15
dew_prof_k = dew_prof+273.15
def _lcl_iter(p, p0, w, t, nan_mask_list):
    td = dewpoint(vapor_pressure(p/100, w)) + 273.15
    p_new = (p0 * (td / t) ** (1. / kappa))
    nan_mask_list[0] = nan_mask_list[0] | np.isnan(p_new)

    return np.where(np.isnan(p_new), p, p_new)

# Handle nans by creating a mask that gets set by our _lcl_iter function if it
# ever encounters a nan, at which point pressure is set to p, stopping iteration.
nan_mask_list = [False]  # Use a mutable list to store the mask
es=saturation_vapor_pressure(dew_prof[0])
w = mixing_ratio(es, pressure_prof[0])
lcl_p = so.fixed_point(_lcl_iter, 
                       pressure_prof_pa[0], 
                       args=(pressure_prof_pa[0], 
                             w, 
                             temp_prof_k[0], 
                             nan_mask_list
                             ),
                        xtol=1e-5,
                        maxiter=50)
lcl_p = np.where(nan_mask_list[0], np.nan, lcl_p)

# np.isclose needed if surface is LCL due to precision error with np.log in dewpoint.
# Causes issues with parcel_profile_with_lcl if removed. Issue #1187
lcl_p = np.atleast_1d(np.where(np.isclose(lcl_p, pressure_prof[0]), pressure_prof[0], lcl_p)/100)
lcl_td = np.atleast_1d(dewpoint(vapor_pressure(lcl_p, w)))
print(lcl_p)
print(lcl_td)

[852.27778167]
[15.82524764]


##### Test on 5d array

In [19]:
data_5d_pa_k = data_5d.copy()
data_5d_pa_k[:,:,:,:,0] = data_5d_pa_k[:,:,:,:,0]*100
data_5d_pa_k[:,:,:,:,1] = data_5d_pa_k[:,:,:,:,1]+273.15
data_5d_pa_k[:,:,:,:,2] = data_5d_pa_k[:,:,:,:,2]+273.15


In [20]:
p_array_pa = data_5d_pa_k[:,:,:,:,0] 
T_array_k = data_5d_pa_k[:,:,:,:,1] 
Td_array_k = data_5d_pa_k[:,:,:,:,2] 

def vectorize_lcl(p, T, Td):
    """Vectorized LCL function based on metpy.calc.lcl. Needs the bottom layer but is dimension agnostic.
       Pressure needs to be in Pa, temperature and dewpoint needs to be in K.
       """
    def _lcl_iter(p, p0, w, t, nan_mask_list):
        td = dewpoint(vapor_pressure(p/100, w)) + 273.15
        p_new = (p0 * (td / t) ** (1. / kappa))
        nan_mask_list[0] = nan_mask_list[0] | np.isnan(p_new)

        return np.where(np.isnan(p_new), p, p_new)

    # Handle nans by creating a mask that gets set by our _lcl_iter function if it
    # ever encounters a nan, at which point pressure is set to p, stopping iteration.
    nan_mask_list = [False]  # Use a mutable list to store the mask
    es=saturation_vapor_pressure(Td-273.15)
    w = mixing_ratio(es, p/100)
    lcl_p = so.fixed_point(_lcl_iter, 
                        p, 
                        args=(p, 
                                w, 
                                T, 
                                nan_mask_list
                                ),
                            xtol=1e-5,
                            maxiter=50)
    lcl_p = np.where(nan_mask_list[0], np.nan, lcl_p)

    # np.isclose needed if surface is LCL due to precision error with np.log in dewpoint.
    # Causes issues with parcel_profile_with_lcl if removed. Issue #1187
    lcl_p = np.where(np.isclose(lcl_p, p), p, lcl_p)/100
    lcl_td = dewpoint(vapor_pressure(lcl_p, w))
    return lcl_p, lcl_td

lcl_p_vectorized, lcl_td_vectorized = vectorize_lcl(p_array_pa[:,:,:,0], T_array_k[:,:,:,0], Td_array_k[:,:,:,0])
lcl_p_vectorized = np.atleast_1d(lcl_p_vectorized)

#### rest of _parcel_profile_helper

In [21]:
def moist_lapse(pressure, temperature):
    def dt(p, t):
        rs = saturation_mixing_ratio(p, t)
        frac = (
            (Rd * t + Lv * rs)
            / (Cp_d + (
                Lv * Lv * rs * epsilon
                / (Rd * t**2)
            ))
        )
        return frac / p

    temperature = np.atleast_1d(temperature)
    pressure = np.atleast_1d(pressure)
    reference_pressure = pressure[0]

    if np.isnan(reference_pressure) or np.all(np.isnan(temperature)):
        return np.full((temperature.size, pressure.size), np.nan)

    pres_decreasing = (pressure[0] > pressure[-1])
    if pres_decreasing:
        # Everything is easier if pressures are in increasing order
        pressure = pressure[::-1]

    # It would be preferable to use a regular solver like RK45, but as of scipy 1.8.0
    # anything other than LSODA goes into an infinite loop when given NaNs for y0.
    solver_args = {'fun': dt, 'y0': temperature,
                   'method': 'LSODA', 'atol': 1e-7, 'rtol': 1.5e-8}

    # Need to handle close points to avoid an error in the solver
    close = np.isclose(pressure, reference_pressure)
    if np.any(close):
        ret = np.broadcast_to(temperature[:, np.newaxis], (temperature.size, np.sum(close)))
    else:
        ret = np.empty((temperature.size, 0), dtype=temperature.dtype)

    # Do we have any points above the reference pressure
    points_above = (pressure < reference_pressure) & ~close
    if np.any(points_above):
        # Integrate upward--need to flip so values are properly ordered from ref to min
        press_side = pressure[points_above][::-1]

        # Flip on exit so t values correspond to increasing pressure
        result = si.solve_ivp(t_span=(reference_pressure, press_side[-1]),
                              t_eval=press_side, **solver_args)
        if result.success:
            ret = np.concatenate((result.y[..., ::-1], ret), axis=-1)
        else:
            raise ValueError('ODE Integration failed. This is likely due to trying to '
                             'calculate at too small values of pressure.')

    # Do we have any points below the reference pressure
    points_below = ~points_above & ~close
    if np.any(points_below):
        # Integrate downward
        press_side = pressure[points_below]
        result = si.solve_ivp(t_span=(reference_pressure, press_side[-1]),
                              t_eval=press_side, **solver_args)
        if result.success:
            ret = np.concatenate((ret, result.y), axis=-1)
        else:
            raise ValueError('ODE Integration failed. This is likely due to trying to '
                             'calculate at too small values of pressure.')

    if pres_decreasing:
        ret = ret[..., ::-1]

    return ret.squeeze()

In [22]:
press_lower = np.concatenate((pressure_prof[pressure_prof >= lcl_p], np.atleast_1d(lcl_p)))
temp_lower = dry_lapse(press_lower, temp_prof_k[0])
# If the pressure profile doesn't make it to the lcl, we can stop here
if np.isclose(np.nanmin(p_array), lcl_p):
    print(press_lower[:-1], lcl_p, units.Quantity(np.array([]), press_lower.units),
            temp_lower[:-1], lcl_td, units.Quantity(np.array([]), temp_lower.units))
else:
    # Establish profile above LCL
    press_upper = np.concatenate((lcl_p, p_array[p_array < lcl_p]))

    # Remove duplicate pressure values from remaining profile. Needed for solve_ivp in
    # moist_lapse. unique will return remaining values sorted ascending.
    unique, indices, counts = np.unique(press_upper, return_inverse=True, return_counts=True)

    # Find moist pseudo-adiabatic profile starting at the LCL, reversing above sorting
    print(unique[::-1], temp_lower[-1])
    temp_upper = moist_lapse(unique[::-1], temp_lower[-1])
    print(temp_upper)
    temp_upper = temp_upper[::-1][indices]

    # # Return profile pieces
    print(press_lower[:-1], lcl_p, press_upper[1:],temp_lower[:-1], lcl_td, temp_upper[1:])

[852.27778167 850.         800.         750.         700.
 650.         600.         550.         500.         450.
 400.         350.         300.         250.         200.
 175.         150.         125.         100.          80.
  70.          60.          50.          40.          30.
  25.          20.        ] 288.97524764465186
[288.97524764 288.93579969 288.04500722 287.1026302  286.10191102
 285.034625   283.89064214 282.6572779  281.31833655 279.85266157
 278.23187285 276.41658274 274.34976131 271.94415228 269.05582969
 267.35611991 265.4200338  263.16543877 260.45679986 257.8024633
 256.23938381 254.45796511 252.3821944  249.88655816 246.73988839
 244.78570661 242.43503747]
[1008.  900.] [852.27778167] [850. 800. 750. 700. 650. 600. 550. 500. 450. 400. 350. 300. 250. 200.
 175. 150. 125. 100.  80.  70.  60.  50.  40.  30.  25.  20.] [303.16795871 293.50872877] [15.82524764] [288.93579969 288.04500722 287.1026302  286.10191102 285.034625
 283.89064214 282.6572779  281.3183365

### Integration:

In [23]:
# Truncate the 4th dimension in data_5d where pressure is less than 50 hPa
# The pressure values are stored in data_5d[:,:,:,:,0]

# Find the index where pressure becomes less than 50 hPa
# Since pressure decreases with height, we need to find the first index where p < 50
threshold_index = np.where(p_array < 100)[0][0]

# Truncate the 4th dimension of data_5d to keep only levels where pressure >= 50 hPa
data_5d = data_5d[:,:,:,:threshold_index,:]

# Print the new shape to confirm
print(f"Shape after truncation: {data_5d.shape}")
print(f"Pressure levels kept: {p_array[:threshold_index]}")

np.trapezoid(data_5d,axis=4).shape


Shape after truncation: (2, 180, 360, 22, 3)
Pressure levels kept: [1008. 1000.  950.  900.  850.  800.  750.  700.  650.  600.  550.  500.
  450.  400.  350.  300.  250.  200.  175.  150.  125.  100.]


(2, 180, 360, 22)

# Final:

In [24]:
mixed_layer_cape_cin(p,T,Td,depth=50*metpy.units.units.hPa)

[900.6037205824425 900.0 850.0 800.0 750.0 700.0 650.0 600.0 550.0 500.0 450.0 400.0 350.0 300.0 250.0 200.0 175.0 150.0 125.0 100.0 80.0 70.0 60.0 50.0 40.0 30.0 25.0 20.0] hectopascal 293.02922746121556 kelvin
temp_upper [293.02922746121556 293.00589484213793 291.00422762193216 288.85100636565124 286.52033059263204 283.97913428290104 281.1844063434547 278.07894493023554 274.5848566984259 270.5932862546365 265.9481338187206 260.42088683848476 253.6758792775867 245.2392400936672 234.52082043863217 220.9223365515545 212.85152462162523 203.7749615726271 193.46771152266598 181.52807471969018 170.31749983738132 163.9421382295467 156.8783693763426 148.91548866559205 139.71765314241844 128.69286750859752 122.16062605523213 114.61531757231914] kelvin
p [1008.0 950.0 900.6037205824425 900.0 850.0 800.0 750.0 700.0 650.0 600.0 550.0 500.0 450.0 400.0 350.0 300.0 250.0 200.0 175.0 150.0 125.0 100.0 80.0 70.0 60.0 50.0 40.0 30.0 25.0 20.0] hectopascal
t [29.464702606075207 25.5 20.95554229358471 

(<Quantity(711.239032, 'joule / kilogram')>,
 <Quantity(-5.48053989, 'joule / kilogram')>)